# dynamo db - create table

In [21]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from boto3.dynamodb.conditions import Key, Attr

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()
    
print("imported modules.")

imported modules.


## dynamodb client

In [22]:
client = boto3.client('dynamodb',
  # endpoint_url = "http://{}:8000".format(creds["host"]),
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            # endpoint_url = "http://{}:8000".format(creds["host"]), \
            region_name='us-east-2')

print("created dynamo client.")

created dynamo client.


## create dynamo table

In [12]:
try:
    table = dynamodb.create_table(
        TableName='tweets',
        KeySchema=[
            {
                'AttributeName': 'user',
                'KeyType': 'HASH'  #Partition key
            },
            {
                'AttributeName': 'created_at',
                'KeyType': 'RANGE'  #Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'user',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'created_at',
                'AttributeType': 'S'
            }

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )

    print("Table status:", table.table_status)

except Exception as e:
    print("EXCEPTION: {}".format(e))

EXCEPTION: An error occurred (ResourceInUseException) when calling the CreateTable operation: Cannot create preexisting table


## wait for table creation

In [13]:
try:
    session = botocore.session.get_session()
    dynamodb_session = session.create_client('dynamodb', \
                        region_name='us-east-2',
                        # endpoint_url = "http://192.168.50.114:8000",
                        aws_access_key_id=creds["aws-access-key"],
                        aws_secret_access_key=creds["aws-secret-access-key"]) # low-level client

    waiter = dynamodb_session.get_waiter('table_exists')
    waiter.wait(TableName="tweets")

except KeyboardInterrupt:
    pass

except Exception as f:
    print("EXCEPTION: {}".format(f))

## scan tweets table

In [16]:
tweets = dynamodb.Table("tweets")

response = tweets.scan()
print("got count of: {}".format(response["Count"]))

got count of: 5583


In [5]:
response

{'Items': [{'inserted_at': '2023-02-05T00:39:29Z',
   'created_at': '2016-04-09T06:18:11Z',
   'favorites': Decimal('0'),
   'retweets': Decimal('0'),
   'user': 'ihearttyou2',
   'id': Decimal('718684415396814849'),
   'tweet': 'Wow thank you for that.'},
  {'inserted_at': '2023-02-05T00:39:29Z',
   'created_at': '2016-04-14T22:58:35Z',
   'favorites': Decimal('0'),
   'retweets': Decimal('0'),
   'user': 'ihearttyou2',
   'id': Decimal('720748114005663745'),
   'tweet': "this is why I don't try with certain people...they just never respond."},
  {'inserted_at': '2023-02-05T00:39:29Z',
   'created_at': '2016-04-14T23:41:01Z',
   'favorites': Decimal('2'),
   'retweets': Decimal('0'),
   'user': 'ihearttyou2',
   'id': Decimal('720758789704650752'),
   'tweet': 'sometimes I miss you. but the less I think about it the better off I am...yet I am still wondering. FUCK'},
  {'inserted_at': '2023-02-05T00:39:29Z',
   'created_at': '2016-04-15T01:35:28Z',
   'favorites': Decimal('0'),
   're

In [1]:
! aws dynamodb list-tables
# --endpoint-url http://xanaxprincess.asuscomm.com:8000

{
    "TableNames": [
        "tweets"
    ]
}


In [5]:
! aws dynamodb describe-table --table-name "tweets" --region "us-east-2"
# --endpoint-url http://xanaxprincess.asuscomm.com:8000

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "created_at",
                "AttributeType": "S"
            },
            {
                "AttributeName": "user",
                "AttributeType": "S"
            }
        ],
        "TableName": "tweets",
        "KeySchema": [
            {
                "AttributeName": "user",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "created_at",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1675471937.919,
        "ProvisionedThroughput": {
            "LastDecreaseDateTime": 1675566252.768,
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 10
        },
        "TableSizeBytes": 14607175,
        "ItemCount": 75389,
        "TableArn": "arn:aws:dynamodb:us-east-2:480687964986:table/tweets",
        "T

In [4]:
# ! aws dynamodb describe-table --table-name "tweets" --endpoint-url http://xanaxprincess.asuscomm.com:8000